In [1]:
import numpy as np

In [2]:
BERTH = 10
p_c = 0.2
p_t = 0.5
# V = np.zeros(BERTH)
REVENUE_C = 0.1
REVENUE_T = 0.3
PUNISHMENT_C = 0.2
PUNISHMENT_T = 0.4

In [5]:
action = np.ones((BERTH + 1, BERTH + 1))
r = np.zeros((BERTH + 1, BERTH + 1, 2))
p = np.zeros((BERTH + 1, BERTH + 1, 2))

In [6]:
def discharge(y):
  z = REVENUE_C - (REVENUE_C - REVENUE_T) * y / BERTH
  return z

# def initial_a():
#     for x in range(BERTH):
#         for y in range(BERTH):
#             if x + y == BERTH and y > 0:
#                 action[x,y] = 1
#             else:
#                 action[x,y] = 3

def initial_r():
    for x in range(BERTH):
        for y in range(BERTH):
            for a in range(1):
                if x + y < BERTH and a == 0:
                    r[x,y,a] = p_c * REVENUE_C + p_t * REVENUE_T
                elif x + y == BERTH and y >= 0 and a == 0:
                    r[x,y,a] = p_c * (-PUNISHMENT_C) + p_t * (-PUNISHMENT_T)
                elif x + y == BERTH and y > 0 and a == 1:
                    r[x,y,a] = p_c * REVENUE_C + p_t * REVENUE_T - discharge(y)
                    
def initial_p():
    for x in range(BERTH):
        for y in range(BERTH):
            for a in range(1):
                if x + y < BERTH:
                    p[x+1,y,a] = p_c
                    p[x,y+1,a] = p_t
                    p[x-1,y,a] = MU * x
                    p[x,y-1,a] = MU * y
                    p[x,  y,a] = 1 - p_c - p_t - MU * x - MU * y
                elif x + y == BERTH and y == 0:
                    p[x-1,y,a] = MU * x
                    p[x  ,y,a] = 1 - MU * x
                elif x + y == BERTH and y > 0:
                    p[x+1,y-1,a] = p_c * a
                    p[x-1,  y,a] = MU * x
                    p[x  ,y-1,a] = MU * y
                    p[x  ,y  ,a] = 1- p_c * a - MU * x - MU * y

In [ ]:
def value_iteration(p_c, p_t, theta=0.0001, discount_factor=1.0):
    
    def one_step_likeahead(V, x, y):
        A = np.zeros(2)
        for a in range(1):
            A[a] = r[x,y,a] + discount_factor * p[x,y,a] * V[x,y,a]
        return A
    
    V = np.zeros((BERTH + 1, BERTH + 1))
    
    while True:
        delta = 0
        for x in range(BERTH):
            for y in range(BERTH):
                A = one_step_likeahead(V, x, y)
                best_action = max(A)
                delta = max(delta, np.abs(best_action - V[x,y]))
                V[x,y] = best_action
        if delta < theta:
            break
    
#     policy = np.zeros((BERTH + 1, BERTH + 1))
    for x in range(BERTH):
        for y in range(BERTH):
            if x + y == BERTH and y > 0:
                A = one_step_likeahead(V, x, y)
                if np.argmax(A) == 0:
                    action[x][y] = 3
                else:
                    action[x][y] = 2